This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
#API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [27]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
data_URL_test = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2020-05-08&end_date=2020-05-08'
res_test = requests.get(data_URL_test)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
if res_test: 
    print(res_test.text)

{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2020-05-08","end_date":"2020-05-08","frequency":"daily","data":[["2020-05-08",96.35,97.35,95.15,95.3,null,134071.0,12869764.45,null,null,null]],"collapse":null,"order":null}}


In [28]:
json_test = res_test.json()

In [6]:
json_test['dataset_data']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [7]:
json_test['dataset_data']['data']

[['2020-05-08',
  96.35,
  97.35,
  95.15,
  95.3,
  None,
  134071.0,
  12869764.45,
  None,
  None,
  None]]

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [12]:
data_URL = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31'

In [13]:
res = requests.get(data_URL)
json = res.json()

### 2. Convert the returned JSON object into a Python dictionary.

In [42]:
#Dictionary contains each column name as key with its values as list of all values
'''
{'Date': ['2020-05-08', '2020-05-07',...],
 'Open': [96.35, 94.55,...],
 'High': [97.35, 97.25,...],
 'Low': [95.15, 94.25,...],
 'Close': [95.3, 96.0,...],
 'Change': [None, None,...],
 'Traded Volume': [134071.0, 210096.0,...],
 'Turnover': [12869764.45, 20200879.55,...],
 'Last Price of the Day': [None, None,...],
 'Daily Traded Units': [None, None,...],
 'Daily Turnover': [None, None,...]}
'''
afx_dict = {}
for i, val in enumerate(json['dataset_data']['column_names']):
    afx_dict[val] = []
    {afx_dict[val].append(v[i]) for v in json['dataset_data']['data']}    

### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [34]:
#Get a dict of all opening prices
dict_open = dict(zip(afx_dict['Date'],afx_dict['Open']))

In [35]:
#For each open value, if value is None, send 0 to max function [Comparision of None with float will fail]
max_open = max(dict_open.values(),key=lambda x: 0.0 if x is None else x)

#Use generator to filter float values and feed them to min function
min_open = min((x for x in dict_open.values() if x is not None))

In [36]:
print('Highest value: ' + str(max_open))
print('Lowest value: ' + str(min_open))

Highest value: 53.11
Lowest value: 34.0


### 4. What was the largest change in any one day (based on High and Low price)?

In [68]:
#Generate a list of max_value - min value for each day
#the lambda function accepts 2 variable and produces their difference
#map applies this function to each value in afx_dict['High'] and afx_dict['Low']


list_change = list(map(lambda m, n: m-n, afx_dict['High'], afx_dict['Low']))

In [69]:
max(list_change)

2.8100000000000023

### 5. What was the largest change between any two days (based on Closing Price)?

In [39]:
#zip list of all closing values <---> list of all values starting from 2nd element
#Calculate abs difference for each pair

list_close = [abs(j-i) for i,j in zip(afx_dict['Close'], afx_dict['Close'][1:])]

In [40]:
max(list_close)

2.559999999999995

### 6. What was the average daily trading volume during this year?

In [41]:
avg = sum(afx_dict['Traded Volume'])/len(afx_dict['Traded Volume'])
print(avg)

89124.33725490196


### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

Excellence: Student solves the 7th optional task, uses list comprehensions in their code,
and abstracts certain pieces of the solutions into separate functions (i.e. extracting the index of a data field).


In [49]:
#obtain a new sorted list of all trade volumes
sorted_trade = sorted(afx_dict['Traded Volume'])

In [50]:
def find_median(l):
    sort_l = sorted(l)
    n = len(sort_l)
    if n % 2 == 0:
        m = (l[n//2] + l[n//2 - 1])/2
    else:
        m = l[n//2]
    return m

In [51]:
med = find_median(sorted_trade)

In [52]:
med

76286.0

In [ ]:
[afx_dict['Traded Volume'][i] for i in sorted_trade)]

In [72]:
l = len(sorted_trade)
sorted_trade.index(sorted_trade[l//2])

127